In [13]:
function update_values(self::Any, x::Float64, y::Float64, yaw::Float64, speed::Float64, timestamp::Float64, frame::Any)
    self._current_x = x
    self._current_y = y
    self._current_yaw = yaw
    self._current_speed = speed
    self._current_timestamp = timestamp
    self._current_frame = frame
    if self._current_frame != nothing
        self._start_control_loop = true
    end
end

function update_desired_speed(self::Any)
    min_idx = 0
    min_dist = Inf
    desired_speed = 0
    for i in 1:length(self._waypoints)
        dist = norm([self._waypoints[i][1] - self._current_x, self._waypoints[i][2] - self._current_y])
        if dist < min_dist
            min_dist = dist
            min_idx = i
        end
    end
    if min_idx < length(self._waypoints)
        desired_speed = self._waypoints[min_idx][3]
    else
        desired_speed = self._waypoints[end][3]
    end
    self._desired_speed = desired_speed
end

function update_waypoints(self::Any, new_waypoints::Array)
    self._waypoints = new_waypoints
end

function get_commands(self::Any)
    return self._set_throttle, self._set_steer, self._set_brake
end

function set_throttle(self::Any, input_throttle::Float64)
    throttle = max(min(input_throttle, 1.0), 0.0)
    self._set_throttle = throttle
end

function set_steer(self::Any, input_steer_in_rad::Float64)
    # Convert radians to [-1, 1]
    input_steer = self._conv_rad_to_steer * input_steer_in_rad
    # Clamp the steering command to valid bounds
    steer = max(min(input_steer, 1.0), -1.0)
    self._set_steer = steer
end

function set_brake(self::Any, input_brake::Float64)
    # Clamp the steering command to valid bounds
    brake = max(min(input_brake, 1.0), 0.0)
    self._set_brake = brake
end

function update_controls(self::Any)
    x = self._current_x
    y = self._current_y
    yaw = self._current_yaw
    v = self._current_speed
    update_desired_speed(self)
    v_desired = self._desired_speed
    t = self._current_timestamp
    waypoints = self._waypoints
    throttle_output = 0
    steer_output = 0
    brake_output = 0

    if self._start_control_loop
        self.vars.create_var("v_previous", 0.0)
        self.vars.create_var("I", 0.0)
        self.vars.create_var("t_prev", -100)
        self.vars.create_var("e_prev", 0)

        KP = 2
        KI = 0.1
        KD = 2

        e = v_desired - v
        P = KP * e
        self.vars.I =  self.vars.I + KI * e * (t - self.vars.t_prev)
        D = KD * (e - self.vars.e_prev) / (t - self.vars.t_prev)

        u_out = P + self.vars.I + D

        if u_out >= 0
            throttle_output = u_out
            brake_output = 0
        else
            throttle_output = 0
            brake_output = -u_out
        end

self.set_throttle(throttle_output)  # in percent (0 to 1)
self.set_brake(brake_output)        # in percent (0 to 1)


LoadError: invalid redefinition of constant Controller2D

In [26]:
using LinearAlgebra
using Plots

# Define bicycle model function
function bicycle_model(state, u, dt, L)
    x, y, θ, v = state
    δ = u[1]
    a = u[2]
    β = atan((1/L) * tan(δ))
    x += v * cos(θ + β) * dt
    y += v * sin(θ + β) * dt
    θ += (v/L) * sin(β) * dt
    v += a * dt
    return [x, y, θ, v]
end

# Define PID controller function
function pid_controller(kp, ki, kd, Ts, y_ref, y_init)
    integral_error = 0.0
    previous_error = 0.0
    y = y_init
    y_traj = [y]
    t_traj = [0.0]
    for i = 1:length(y_ref)
        error = y_ref[i] - y
        integral_error += error * Ts
        derivative_error = (error - previous_error) / Ts
        u = kp * error + ki * integral_error + kd * derivative_error
        previous_error = error
        δ = u
        a = 0.5
        state = [0.0, 0.0, 0.0, 5.0] # initial state of the bicycle model
        L = 1.5 # bicycle wheelbase
        state = bicycle_model(state, [δ, a], Ts, L) # propagate state using bicycle model
        y = state[2] # use lateral position (y) as output variable for controller
        push!(y_traj, y)
        push!(t_traj, i * Ts)
    end
    return t_traj, y_traj
end

# Define reference trajectory (sinusoidal path)
t = 0:0.1:10
y_ref = 2 * sin.(t)

# Set PID gains and sampling time
kp = 1.0
ki = 0.1
kd = 0.01
Ts = 0.1

# Run PID controller with initial condition of 0
t_traj, y_traj = pid_controller(kp, ki, kd, Ts, y_ref, 0)

# Plot results
plot(t, y_ref, label="Reference Trajectory")
plot!(t_traj, y_traj, label="System Response")
xlabel!("Time (s)")
ylabel!("Lateral Position (m)")


LoadError: InexactError: Int64(0.07427717489574286)

In [22]:
function pid_controller(kp, ki, kd, Ts, y_ref, y_init::Float64)
    integral_error = 0
    previous_error = 0
    y = y_init
    y_traj = [y]
    t_traj = [0]
    for i = 1:length(y_ref)
        error = y_ref[i] - y
        integral_error += error * Ts
        derivative_error = (error - previous_error) / Ts
        u = kp * error + ki * integral_error + kd * derivative_error
        y += u * Ts
        previous_error = error
        push!(y_traj, y)
        push!(t_traj, (i*Ts)) # Round Ts to 3 decimal places before adding to t_traj
    end
    return t_traj, y_traj
end

# Set PID gains and sampling time
kp = 1.0
ki = 0.1
kd = 0.01
Ts = 0.01

# Set reference trajectory as a sin wave
t_ref = collect(0:Ts:10)
y_ref = sin.(t_ref)

# Run PID controller with initial condition of 0
y_init = 0.0
t_traj, y_traj = pid_controller(kp, ki, kd, Ts, y_ref, y_init)

# Plot results
using Plots
plot(t_ref, y_ref, label="Reference Trajectory")
plot!(t_traj, y_traj, label="System Response")
xlabel!("Time (s)")
ylabel!("Amplitude")


LoadError: InexactError: Int64(0.01)